In [163]:
import time
import numpy as np  # data vectors
import pandas as pd # data analysis
from collections import Counter
from pandas.io.json import json_normalize # transform json into pandas dataframe
import matplotlib.cm as cm #plotting
import matplotlib.colors as colors
import matplotlib.pyplot as py
import json
import requests
from geopy.geocoders import Nominatim #get lat and long
from sklearn.cluster import KMeans #clustering
import folium #visualise map

### 1.Create Dataframe 

In [164]:

CSV = pd.read_csv("Battle of Neighborhoods/ThamesValleyCRIMEDATA.csv")

In [165]:
CSV.head()

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Count
0,3d294010dbca88ade8b95964f03d79dae86cbece156e32...,2019-08,Thames Valley Police,Thames Valley Police,-1.327884,51.753488,On or near Eynsham Road,E01028708,Vale of White Horse 001A,Violence and sexual offences,1
1,dbcbf4f976221e1e202c7019f2803f9ba80a8e1c8881d9...,2019-08,Thames Valley Police,Thames Valley Police,-1.308601,51.748808,On or near Third Acre Rise,E01028709,Vale of White Horse 001B,Burglary,1
2,95569239a93eb375ef1a30f147975303c0aaa322755be2...,2019-08,Thames Valley Police,Thames Valley Police,-1.312464,51.750027,On or near Grange Court,E01028709,Vale of White Horse 001B,Violence and sexual offences,1
3,cdb82cca5ab21305455295afad2e04a4f2b4b2066d2709...,2019-08,Thames Valley Police,Thames Valley Police,-1.307831,51.741359,On or near Barn Close,E01028710,Vale of White Horse 001C,Other theft,1
4,f94e1c275753292c47a748c7241c75beb667817009ef96...,2019-08,Thames Valley Police,Thames Valley Police,-1.308480,51.742487,On or near Delamare Way,E01028710,Vale of White Horse 001C,Public order,1


In [230]:
df_abiC = CSV[CSV['Location'].str.contains('Abingdon')].reset_index(drop=True)
print(df_abiC['LSOA name'].value_counts())
df_abiC.head()


Vale of White Horse 002A    12
Vale of White Horse 010C    12
Vale of White Horse 010A     5
Vale of White Horse 007A     4
Vale of White Horse 010E     3
Vale of White Horse 001C     3
Name: LSOA name, dtype: int64


'Vale of White Horse 002A'

#### Use geopy library to get the latitude and longitude values of Abingdon.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>abi_explorer</em>, as shown below.

In [206]:
address = 'Abingdon-on-Thames, United Kingdom'

geolocator = Nominatim(user_agent="abi_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Abingdon-on-Thames, United Kingdom are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Abingdon-on-Thames, United Kingdom are 51.6714842, -1.2779715.


In [156]:
Map2 = CSV.groupby(['Latitude','Longitude']).size().reset_index().values.tolist()
Map2[:2]

[[51.542968, -1.2570000000000001, 3.0], [51.555739, -1.511144, 3.0]]

 Generate maps to visualize the neighborhood of interest and how they cluster together

In [ ]:
boundary_json = json.load(open('thames-valley.geojson'))


In [283]:
m = folium.Map(location=[41.20, -77.50], tiles="cartodbdark_matter", zoom_start=8.3)

theft = folium.Choropleth(
    geo_data=boundary_json,
    data=df_abiC,               
    columns=['county_name', 'rate'],
    key_on='feature.properties.county_name',
    fill_color='OrRd',
    fill_opacity=0.9,
    nan_fill_color='#ffffff',
    nan_fill_opacity=0.9,
    legend_name='Incident rate per 100,000 people',
    highlight=True,
    name='Theft'
).add_to(m)

folium.GeoJson(
    boundart_json,
    tooltip=folium.features.GeoJsonTooltip(fields=['feature.properties.county_name'])
).add_to(theft)


folium.LayerControl().add_to(m) 
m.save('Crime_Map.html')

print('Map created.')

AttributeError: module 'folium' has no attribute 'Choropleth'

In [222]:
from folium.plugins import HeatMap

m2 = folium.Map(Map2[0][:2], zoom_start=10)

HeatMap(Map2).add_to(m2)
m2

In [223]:
CLIENT_ID = 'GOI03BVANIMQZ54ZM0T5XJTFQSHWG43CDOWVLGVKDZFRXMUA' # your Foursquare ID
CLIENT_SECRET = 'G5CADWKCNYVXYY54Z1ARDUINRHBAKSCAHFCV3HMPH00BVUZH' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GOI03BVANIMQZ54ZM0T5XJTFQSHWG43CDOWVLGVKDZFRXMUA
CLIENT_SECRET:G5CADWKCNYVXYY54Z1ARDUINRHBAKSCAHFCV3HMPH00BVUZH


#### Define Foursquare Credentials and Version

In [231]:
df_abiC.loc[0, 'LSOA name']

'Vale of White Horse 002A'

In [232]:
neighborhood_latitude = df_abiC.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_abiC.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_abiC.loc[0, 'LSOA name'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Vale of White Horse 002A are 51.729093, -1.254532.


In [233]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=GOI03BVANIMQZ54ZM0T5XJTFQSHWG43CDOWVLGVKDZFRXMUA&client_secret=G5CADWKCNYVXYY54Z1ARDUINRHBAKSCAHFCV3HMPH00BVUZH&v=20180604&ll=51.729093,-1.254532&radius=500&limit=100'

In [234]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f92134460d0572b91e17459'},
 'response': {'headerLocation': 'South Hinksey',
  'headerFullLocation': 'South Hinksey',
  'headerLocationGranularity': 'city',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 51.7335930045, 'lng': -1.2472802335730881},
   'sw': {'lat': 51.7245929955, 'lng': -1.2617837664269118}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4dd8e933d22d38ef430c6a32',
       'name': 'GO Outdoors',
       'location': {'address': '426 Abingdon Road',
        'lat': 51.7316112,
        'lng': -1.2546855,
        'labeledLatLngs': [{'label': 'display',
          'lat': 51.7316112,
          'lng': -1.2546855}],
        'distance': 280,
        'postalCode': 'OX1 4XN',
        'cc': 'GB',
        'city': 'Oxford',
 

In [235]:
# function that extracts the category of the venue from Foursqure json response
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [236]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-236-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,GO Outdoors,Outdoor Supply Store,51.731611,-1.254685
1,Travelodge,Hotel,51.731489,-1.249284
2,Camping and Caravanning Club Site,Campground,51.732062,-1.251512
3,Redbridge Park & Ride,Parking,51.729640,-1.248733


In [237]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


In [238]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['LSOA name', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [240]:
df_abiC = CSV[CSV['Location'].str.contains('Abingdon')].reset_index(drop=True)

abi_venues = getNearbyVenues(names=df_abiC['LSOA name'],
                                   latitudes=df_abiC['Latitude'],
                                   longitudes=df_abiC['Longitude']
                                  )

Vale of White Horse 002A
Vale of White Horse 010A
Vale of White Horse 007A
Vale of White Horse 007A
Vale of White Horse 010A
Vale of White Horse 010A
Vale of White Horse 010C
Vale of White Horse 007A
Vale of White Horse 010A
Vale of White Horse 010C
Vale of White Horse 010C
Vale of White Horse 010E
Vale of White Horse 001C
Vale of White Horse 001C
Vale of White Horse 010C
Vale of White Horse 010C
Vale of White Horse 010C
Vale of White Horse 002A
Vale of White Horse 002A
Vale of White Horse 002A
Vale of White Horse 010C
Vale of White Horse 010C
Vale of White Horse 010C
Vale of White Horse 002A
Vale of White Horse 002A
Vale of White Horse 010E
Vale of White Horse 007A
Vale of White Horse 010C
Vale of White Horse 010E
Vale of White Horse 002A
Vale of White Horse 002A
Vale of White Horse 002A
Vale of White Horse 010C
Vale of White Horse 010C
Vale of White Horse 001C
Vale of White Horse 002A
Vale of White Horse 002A
Vale of White Horse 002A
Vale of White Horse 010A


In [242]:
print(abi_venues.shape)
abi_venues.head()

(132, 7)


,LSOA name,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Vale of White Horse 002A,51.729093,-1.254532,GO Outdoors,51.731611,-1.254685,Outdoor Supply Store
1,Vale of White Horse 002A,51.729093,-1.254532,Travelodge,51.731489,-1.249284,Hotel
2,Vale of White Horse 002A,51.729093,-1.254532,Camping and Caravanning Club Site,51.732062,-1.251512,Campground
3,Vale of White Horse 002A,51.729093,-1.254532,Redbridge Park & Ride,51.729640,-1.248733,Parking
4,Vale of White Horse 010A,51.647852,-1.311395,Keyminders,51.648870,-1.306261,Pet Service


In [254]:
abi_venues.groupby('LSOA name').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
LSOA name,,,,,,
Vale of White Horse 001C,8,8,8,8,8,8
Vale of White Horse 002A,48,48,48,48,48,48
Vale of White Horse 007A,6,6,6,6,6,6
Vale of White Horse 010A,13,13,13,13,13,13
Vale of White Horse 010C,48,48,48,48,48,48
Vale of White Horse 010E,9,9,9,9,9,9


In [276]:
print('There are {} uniques categories.'.format(len(abi_venues['Venue Category'].unique())))

There are 18 uniques categories.


In [277]:
# one hot encoding
abi_onehot = pd.get_dummies(abi_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
abi_onehot['LSOA name'] = abi_venues['LSOA name'] 

# move neighborhood column to the first column
fixed_columns = [abi_onehot.columns[-1]] + list(abi_onehot.columns[:-1])
abi_onehot = abi_onehot[fixed_columns]

abi_onehot.head()

,LSOA name,Campground,Canal Lock,Construction & Landscaping,Cricket Ground,Cupcake Shop,Food & Drink Shop,Gastropub,Golf Course,Grocery Store,Hotel,Indian Restaurant,Outdoor Supply Store,Parking,Pet Service,Pub,Seafood Restaurant,Spa,Sporting Goods Shop
0,Vale of White Horse 002A,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,Vale of White Horse 002A,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,Vale of White Horse 002A,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Vale of White Horse 002A,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,Vale of White Horse 010A,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [278]:
abi_grouped = abi_onehot.groupby('LSOA name').mean().reset_index()
abi_grouped

,LSOA name,Campground,Canal Lock,Construction & Landscaping,Cricket Ground,Cupcake Shop,Food & Drink Shop,Gastropub,Golf Course,Grocery Store,Hotel,Indian Restaurant,Outdoor Supply Store,Parking,Pet Service,Pub,Seafood Restaurant,Spa,Sporting Goods Shop
0,Vale of White Horse 001C,0.00,0.000000,0.000000,0.25,0.125,0.000000,0.25,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.375,0.000000,0.000000,0.0
1,Vale of White Horse 002A,0.25,0.000000,0.000000,0.00,0.000,0.000000,0.00,0.0,0.000000,0.250000,0.000000,0.25,0.25,0.000000,0.000,0.000000,0.000000,0.0
2,Vale of White Horse 007A,0.00,0.000000,0.000000,0.00,0.000,0.000000,0.00,0.5,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000,0.000000,0.000000,0.5
3,Vale of White Horse 010A,0.00,0.000000,0.307692,0.00,0.000,0.307692,0.00,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.384615,0.000,0.000000,0.000000,0.0
4,Vale of White Horse 010C,0.00,0.000000,0.000000,0.00,0.000,0.000000,0.00,0.0,0.041667,0.208333,0.041667,0.00,0.00,0.000000,0.500,0.000000,0.208333,0.0
5,Vale of White Horse 010E,0.00,0.666667,0.000000,0.00,0.000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000,0.333333,0.000000,0.0


In [279]:
num_top_venues = 5

for hood in abi_grouped['LSOA name']:
    print("----"+hood+"----")
    temp = abi_grouped[abi_grouped['LSOA name'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Vale of White Horse 001C----
            venue  freq
0             Pub  0.38
1  Cricket Ground  0.25
2       Gastropub  0.25
3    Cupcake Shop  0.12
4      Campground  0.00


----Vale of White Horse 002A----
                  venue  freq
0            Campground  0.25
1  Outdoor Supply Store  0.25
2               Parking  0.25
3                 Hotel  0.25
4          Cupcake Shop  0.00


----Vale of White Horse 007A----
                 venue  freq
0  Sporting Goods Shop   0.5
1          Golf Course   0.5
2    Indian Restaurant   0.0
3                  Spa   0.0
4   Seafood Restaurant   0.0


----Vale of White Horse 010A----
                        venue  freq
0                 Pet Service  0.38
1  Construction & Landscaping  0.31
2           Food & Drink Shop  0.31
3                  Campground  0.00
4           Indian Restaurant  0.00


----Vale of White Horse 010C----
               venue  freq
0                Pub  0.50
1              Hotel  0.21
2                Spa  0.21
3  In

In [280]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [281]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['LSOA name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['LSOA name'] = abi_grouped['LSOA name']

for ind in np.arange(abi_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(abi_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,LSOA name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Vale of White Horse 001C,Pub,Cricket Ground,Gastropub,Cupcake Shop,Sporting Goods Shop,Golf Course,Canal Lock,Construction & Landscaping,Food & Drink Shop,Grocery Store
1,Vale of White Horse 002A,Campground,Parking,Outdoor Supply Store,Hotel,Gastropub,Canal Lock,Construction & Landscaping,Cricket Ground,Cupcake Shop,Food & Drink Shop
2,Vale of White Horse 007A,Sporting Goods Shop,Golf Course,Spa,Canal Lock,Construction & Landscaping,Cricket Ground,Cupcake Shop,Food & Drink Shop,Gastropub,Grocery Store
3,Vale of White Horse 010A,Pet Service,Construction & Landscaping,Food & Drink Shop,Sporting Goods Shop,Golf Course,Canal Lock,Cricket Ground,Cupcake Shop,Gastropub,Grocery Store
4,Vale of White Horse 010C,Pub,Hotel,Spa,Grocery Store,Indian Restaurant,Gastropub,Canal Lock,Construction & Landscaping,Cricket Ground,Cupcake Shop


In [265]:
# set number of clusters
kclusters = 5

abi_grouped_clustering = abi_grouped.drop('LSOA name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(abi_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 1, 4, 0, 2], dtype=int32)

In [272]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

abi_merged = df_abiC

# merge toronto_grouped with df_toronto to add latitude/longitude for each neighborhood
abi_merged = abi_merged.join(neighborhoods_venues_sorted.set_index('LSOA name'), on='LSOA name')

abi_merged.head() # check the last columns!

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,f805a544e8187ec1bda841171bd8c93d74f81547d0a1af...,2019-08,Thames Valley Police,Thames Valley Police,-1.254532,51.729093,On or near Abingdon Road,E01028733,Vale of White Horse 002A,Other theft,...,Campground,Parking,Outdoor Supply Store,Hotel,Gastropub,Canal Lock,Construction & Landscaping,Cricket Ground,Cupcake Shop,Food & Drink Shop
1,da628637b46f0cad201074d3f587a2c13539999e809fb6...,2019-08,Thames Valley Police,Thames Valley Police,-1.311395,51.647852,On or near Abingdon Road,E01028714,Vale of White Horse 010A,Shoplifting,...,Pet Service,Construction & Landscaping,Food & Drink Shop,Sporting Goods Shop,Golf Course,Canal Lock,Cricket Ground,Cupcake Shop,Gastropub,Grocery Store
2,479033dccfaea7ebe19f463b2eb5f6a13a5a6a10ebdf35...,2019-09,Thames Valley Police,Thames Valley Police,-1.370958,51.685321,On or near Abingdon Road,E01028726,Vale of White Horse 007A,Vehicle crime,...,Sporting Goods Shop,Golf Course,Spa,Canal Lock,Construction & Landscaping,Cricket Ground,Cupcake Shop,Food & Drink Shop,Gastropub,Grocery Store
3,399c4f8e4b309817573c37361155d7a0da30119790b567...,2019-09,Thames Valley Police,Thames Valley Police,-1.370958,51.685321,On or near Abingdon Road,E01028726,Vale of White Horse 007A,Violence and sexual offences,...,Sporting Goods Shop,Golf Course,Spa,Canal Lock,Construction & Landscaping,Cricket Ground,Cupcake Shop,Food & Drink Shop,Gastropub,Grocery Store
4,b132fe1dd79abda0e48c1c85d170b30253930aeb9bb7ea...,2019-09,Thames Valley Police,Thames Valley Police,-1.310317,51.649321,On or near Abingdon Road,E01028714,Vale of White Horse 010A,Burglary,...,Pet Service,Construction & Landscaping,Food & Drink Shop,Sporting Goods Shop,Golf Course,Canal Lock,Cricket Ground,Cupcake Shop,Gastropub,Grocery Store


In [273]:
print('There are {} uniques categories.'.format(len(abi_merged['Cluster Labels'].unique())))

There are 5 uniques categories.


In [275]:
#create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_abiC['Latitude'], df_abiC['Longitude'], df_abiC['LSOA name'], df_abiC['Location']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

TypeError: unsupported operand type(s) for -: 'str' and 'int'

In [214]:
crime_types = MergeBoth[['name','type']].drop_duplicates().set_index('name')['type'].to_dict()
crime_types


{'Anti-social behaviour': 'anti-social-behaviour',
 'Bicycle theft': 'bicycle-theft',
 'Burglary': 'burglary',
 'Criminal damage and arson': 'criminal-damage-arson',
 'Other theft': 'other-theft',
 'Public order': 'public-order',
 'Shoplifting': 'shoplifting',
 'Vehicle crime': 'vehicle-crime',
 'Violence and sexual offences': 'violent-crime',
 'Other crime': 'other-crime',
 'Drugs': 'drugs',
 'Theft from the person': 'theft-from-the-person'}

#### Define Foursquare Credentials and Version

In [98]:
dates_Years = [d for d in api.get_dates() if '2018' or '2019' in d]
dates_Years

['2020-08',
 '2020-07',
 '2020-06',
 '2020-05',
 '2020-04',
 '2020-03',
 '2020-02',
 '2020-01',
 '2019-12',
 '2019-11',
 '2019-10',
 '2019-09',
 '2019-08',
 '2019-07',
 '2019-06',
 '2019-05',
 '2019-04',
 '2019-03',
 '2019-02',
 '2019-01',
 '2018-12',
 '2018-11',
 '2018-10',
 '2018-09',
 '2018-08',
 '2018-07',
 '2018-06',
 '2018-05',
 '2018-04',
 '2018-03',
 '2018-02',
 '2018-01',
 '2017-12',
 '2017-11',
 '2017-10',
 '2017-09']

In [101]:
crimes_All= pd.DataFrame()

for d in dates_Years:
    tmp = api.get_crimes_area(boundary, date=d)
    crimes_All = pd.concat([crimes_All, setCrimesAsDataFrame(tmp)])
    
crimes_All.head()

,cid,type,month,name,lid,location,lat,lon
0,86061326,anti-social-behaviour,2020-08,Anti-social behaviour,1208644,On or near Glebe Close,51.835287,-0.638018
1,86061004,anti-social-behaviour,2020-08,Anti-social behaviour,1210358,On or near Park/open Space,51.946490,-0.703851
2,86060136,anti-social-behaviour,2020-08,Anti-social behaviour,1209994,On or near Redwood Close,51.897264,-0.718799
3,86061605,anti-social-behaviour,2020-08,Anti-social behaviour,1208635,On or near Maud Jane's Close,51.838668,-0.631864
4,86061968,anti-social-behaviour,2020-08,Anti-social behaviour,1183686,On or near Newton Road,51.957982,-0.765924


In [102]:
crimes_All2= pd.DataFrame()

for d in dates_Years:
    tmp = api.get_crimes_area(boundary2, date=d)
    crimes_All2 = pd.concat([crimes_All2, setCrimesAsDataFrame(tmp)])
    
crimes_All2.head()

,cid,type,month,name,lid,location,lat,lon
0,86060551,anti-social-behaviour,2020-08,Anti-social behaviour,1202422,On or near Southmoor Way,51.677685,-1.291678
1,86059616,anti-social-behaviour,2020-08,Anti-social behaviour,1201206,On or near The Hyde,51.661900,-1.296811
2,86059030,anti-social-behaviour,2020-08,Anti-social behaviour,1201206,On or near The Hyde,51.661900,-1.296811
3,86059277,anti-social-behaviour,2020-08,Anti-social behaviour,1201206,On or near The Hyde,51.661900,-1.296811
4,86059090,anti-social-behaviour,2020-08,Anti-social behaviour,1201206,On or near The Hyde,51.661900,-1.296811


In [104]:
Todos_crimes=pd.concat([crimes_All,crimes_All2],axis=0)

In [112]:
overall_data = []

for d in dates_Years:
    tmp_crimes = api.get_crimes_area(boundary, date=d)
    df = setCrimesAsDataFrame(tmp_crimes)
    MainsDB = MergeBoth.groupby(['lat','lon']).size().reset_index().values.tolist()
    overall_data.append(MainsDB)

In [111]:
from folium.plugins import HeatMapWithTime

m = folium.Map(overall_data[0][0][:2], tiles='stamentoner', zoom_start=13)

HeatMapWithTime(overall_data).add_to(m)

m

In [115]:
#For a particular crime type, get the count of crimes by location
crimes = setCrimesAsDataFrame( api.get_crimes_area(boundary, date='2019-01') )
public_order_crime = crimes[crimes['type']=='public-order']
public_order_crime_count_by_location = public_order_crime.groupby(['lat','lon']).size().to_frame('Count').reset_index()
public_order_crime_count_by_location.head()

,lat,lon,Count
0,51.835283,-0.632315,2
1,51.887907,-0.690498,1
2,51.897264,-0.718799,1
3,51.897377,-0.720918,1
4,51.899269,-0.718742,1


In [116]:
#For a particular crime type, get the count of crimes by location
crimes2 = setCrimesAsDataFrame( api.get_crimes_area(boundary2, date='2019-01') )
public_order_crime2 = crimes[crimes['type']=='public-order']
public_order_crime_count_by_location2 = public_order_crime.groupby(['lat','lon']).size().to_frame('Count').reset_index()
public_order_crime_count_by_location2.head()

,lat,lon,Count
0,51.835283,-0.632315,2
1,51.887907,-0.690498,1
2,51.897264,-0.718799,1
3,51.897377,-0.720918,1
4,51.899269,-0.718742,1


In [117]:
#For convenience, let's go back to using the df name...
df = public_order_crime_count_by_location.reset_index()

In [118]:
#https://geopandas.readthedocs.io/en/latest/gallery/create_geopandas_from_pandas.html
import geopandas
from shapely.geometry import Point

df['Coordinates'] = list(zip(df['lon'], df['lat']))
df['Coordinates'] = df['Coordinates'].apply(Point)

df['RCoordinates'] = list(zip(df['lat'], df['lon']))
df['RCoordinates'] = df['RCoordinates'].apply(Point)
gdf = geopandas.GeoDataFrame(df, geometry='Coordinates')

#Avoid ambiguity about whether index is int or string
gdf['index'] = gdf['index'].apply(lambda x: 'a_{}'.format(x))

gdf.head()

,index,lat,lon,Count,Coordinates,RCoordinates
0,a_0,51.835283,-0.632315,2,POINT (-0.63231 51.83528),POINT (51.835283 -0.632315)
1,a_1,51.887907,-0.690498,1,POINT (-0.69050 51.88791),POINT (51.887907 -0.6904979999999999)
2,a_2,51.897264,-0.718799,1,POINT (-0.71880 51.89726),POINT (51.897264 -0.718799)
3,a_3,51.897377,-0.720918,1,POINT (-0.72092 51.89738),POINT (51.897377 -0.7209179999999999)
4,a_4,51.899269,-0.718742,1,POINT (-0.71874 51.89927),POINT (51.899269 -0.718742)


In [132]:
gdf.crs = {'init' :'epsg:4326 '}
gdf.crs

/Users/mavrioz/opt/anaconda3/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


<Geographic 2D CRS: +init=epsg:4326  +type=crs>
Name: WGS 84
Axis Info [ellipsoidal]:
- lon[east]: Longitude (degree)
- lat[north]: Latitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [133]:
#Check how the boundary co-ordinates are presented
boundary[:2]
#Looks like: lat, lon

[(51.71444649741, -1.2352394640321), (51.713839174699, -1.2353148374778)]

In [123]:
from folium.plugins import HeatMap


m = createFoliumMap(gdf)


HeatMap(df).add_to(m)

folium.GeoJson(
    geoJsonFromNeighbourhoodBoundary(boundary),
    name='geojson',
    
).add_to(m)
folium.GeoJson(
    geoJsonFromNeighbourhoodBoundary(boundary2),
    name='geojson',
    
).add_to(m)


m

NameError: name 'createFoliumMap' is not defined